In [1]:
import os
import torch
import sys
sys.path.append('../../lib/exlib/src')

import sys
sys.path.append('../../src')
import sop

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sop.utils.seed_all(42)

# config
exp_config = sop.ImageNetConfig()
val_config = exp_config.get_config('val_sm')
val_config['evaluation']['batch_size'] = 16
val_config

/opt/conda/envs/rapids/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/envs/rapids/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may hav

{'dataset': {'name': 'imagenet_s',
  'root': '/shared_data0/weiqiuy/datasets/imagenet'},
 'training': {'batch_size': 16,
  'num_epochs': 20,
  'mask_batch_size': 64,
  'optimizer': {'name': 'adamw', 'lr': 5e-06, 'weight_decay': 0.01}},
 'evaluation': {'split': 'val', 'num_data': 1, 'batch_size': 16},
 'model': {'type': 'vit',
  'base': 'google/vit-base-patch16-224',
  'sop': '/shared_data0/weiqiuy/sop/exps/imagenet_lr5e-06_tgtnnz0.2_gg0.0600_gs0.0100_ft_identify_fixk_scratch_ks3/best',
  'num_classes': 1000}}

In [2]:
# val_config['model']['sop'] = '/shared_data0/weiqiuy/sop/exps/imagenet_lr5e-06_tgtnnz0.2_gg0.0600_gs0.0100_ft_identify_fixk_scratch_ks3/last'

In [3]:
backbone_model, original_model, processor, backbone_config, model, config = sop.tasks.imagenet.get_model(val_config['model']['type'],
                                                                 backbone_model_name=val_config['model']['base'],
                                                                 backbone_processor_name=val_config['model']['base'],
                                                                 sop_model_name=val_config['model']['sop'], 
                                                                                                         # sop_model_name='/shared_data0/weiqiuy/sop/exps/imagenet_lr5e-06_tgtnnz0.2_gg0.0600_gs0.0100_ft_identify_fixk_scratch_ks3/last',
                                                                                                         eval_mode=True
                                                                                                        )

backbone_model = backbone_model.to(device)
original_model = original_model.to(device)
model = model.to(device)

projection layer is not frozen
projection layer is not frozen
Loaded step 40100


In [4]:
methods = [
    'shap_20',
    'rise_20',
    'lime_20',
    'sop',
    'fullgrad',
    'gradcam',
    'intgrad',
    'attn',
    'archipelago',
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet',
]

In [6]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc
from sop.tasks.images.imagenet import get_explainer

methods = [
    'bcos',
    'xdnn',
    'bagnet',
    'sop',
    'shap_20',
    'rise_20',
    'lime_20',
    'fullgrad',
    'gradcam',
    'intgrad',
    'attn',
    'archipelago',
    'mfaba',
    'agi',
    'ampe',
]


debug = False
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
# method = 'shap_20'
results_all = {}

for method in methods:

    if method == 'sop':
        explainer = model
    else:
        explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)

    method_list = method.split('_')
    explainer_name = method_list[0]

    if len(method_list) == 2:
        suffix = f'_{method_list[1]}'
    else:
        suffix = ''

    if method not in ['sop', 'bagnet']: #, 'xdnn', 'bagnet']:
        ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
    else:
        ATTR_VAL_DATA_DIR = None
    
    val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                              split=val_config['evaluation']['split'], 
                                              num_data=val_config['evaluation']['num_data'],
                                              batch_size=val_config['evaluation']['batch_size'],
                                                            attr_dir=ATTR_VAL_DATA_DIR,
                                              processor=processor, debug=debug)

    # for k in tqdm(np.linspace(0.1, 1, 10)):
    k = 0.2
    results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=True, built_in=True)
    results_all[method] = results
    print(method, results_all[method]['acc'])
    # break

Using cache found in /home/runai-home/.cache/torch/hub/B-cos_B-cos-v2_main


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 919 images and 1000 classes


  0%|          | 0/58 [00:00<?, ?it/s]

bcos 0.04787812840043525


...

agi 0.5875952121871599


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 919 images and 1000 classes


  0%|          | 0/58 [00:00<?, ?it/s]

ampe 0.5179542981501633


In [7]:
# results_path = f'imagenet_acc.pt'

# torch.save(results_all, results_path)

In [5]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc
from sop.tasks.images.imagenet import get_explainer

methods = [
    # 'bcos',
    'xdnn',
    # 'bagnet',
    # 'sop',
    # 'shap_20',
    # 'rise_20',
    # 'lime_20',
    # 'fullgrad',
    # 'gradcam',
    # 'intgrad',
    # 'attn',
    # 'archipelago',
    # 'mfaba',
    # 'agi',
    # 'ampe',
    # 'backbone'
]


debug = False
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
# method = 'shap_20'
results_all = {}

for method in methods:

    if method == 'sop':
        explainer = model
    elif method == 'backbone':
        explainer = original_model
    else:
        explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)

    method_list = method.split('_')
    explainer_name = method_list[0]

    if len(method_list) == 2:
        suffix = f'_{method_list[1]}'
    else:
        suffix = ''

    if method not in ['sop', 'bagnet', 'backbone']: #, 'xdnn', 'bagnet']:
        ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
    else:
        ATTR_VAL_DATA_DIR = None
    
    val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                              split=val_config['evaluation']['split'], 
                                              num_data=val_config['evaluation']['num_data'],
                                              batch_size=val_config['evaluation']['batch_size'],
                                                            attr_dir=ATTR_VAL_DATA_DIR,
                                              processor=processor, debug=debug)

    # for k in tqdm(np.linspace(0.1, 1, 10)):
    k = 0.2
    results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=True, built_in=True)
    results_all[method] = results
    print(method, results_all[method]['acc'])
    # break

  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 919 images and 1000 classes


  0%|          | 0/58 [00:00<?, ?it/s]

xdnn 0.12404787812840043


In [6]:
results_path = f'imagenet_acc_all.pt'

results_load = torch.load(results_path)

In [7]:
for k in results_all:
    results_load[k] = results_all[k]

In [8]:
# torch.save(results_load, 'imagenet_acc_all.pt')
torch.save(results_load, 'imagenet_acc_all2.pt')

In [10]:
for k, v in results_load.items():
    print(k, v['acc'])

bcos 0.04787812840043525
xdnn 0.12404787812840043
bagnet 0.4940152339499456
sop 0.7366702937976061
shap_20 0.6877040261153428
rise_20 0.27747551686615884
lime_20 0.6713819368879217
fullgrad 0.5505984766050055
gradcam 0.545157780195865
intgrad 0.41458106637649617
attn 0.4689880304678999
archipelago 0.4776931447225245
mfaba 0.4929270946681175
agi 0.5875952121871599
ampe 0.5179542981501633
backbone 0.9031556039173014


# Trash

In [8]:
results_all

{'bcos': {'acc': 0.04787812840043525,
  'corrects': [False,
   True,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   True,
   F

In [ ]:
from sop.tasks.images.imagenet import get_explainer

debug = True
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
method = 'shap_20'

if method == 'sop':
    explainer = model
else:
    explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)
    
method_list = method.split('_')
explainer_name = method_list[0]

if len(method_list) == 2:
    suffix = f'_{method_list[1]}'
else:
    suffix = ''

if method != 'sop':
    ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
else:
    ATTR_VAL_DATA_DIR = None
    
val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                          split=val_config['evaluation']['split'], 
                                          num_data=val_config['evaluation']['num_data'],
                                          batch_size=val_config['evaluation']['batch_size'],
                                                        attr_dir=ATTR_VAL_DATA_DIR,
                                          processor=processor, debug=debug)

In [5]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc

results_all = {}
# for k in tqdm(np.linspace(0.1, 1, 10)):
k = 0.2
results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=False, built_in=True)
results_all[method] = results
# break

  0%|          | 0/7 [00:00<?, ?it/s]

SyntaxError: 'break' outside loop (2380042618.py, line 10)

In [6]:
results_all

{'shap_20': {'acc': 0.75,
  'corrects': [False,
   False,
   True,
   False,
   True,
   True,
   False,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True]}}

In [10]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc
from sop.tasks.images.imagenet import get_explainer

methods = [
    # 'shap_20',
    # 'rise_20',
    # 'lime_20',
    # 'fullgrad',
    # 'gradcam',
    # 'intgrad',
    # 'attn',
    # 'archipelago',
    # 'mfaba',
    # 'agi',
    # 'ampe',
    'bcos',
    'xdnn',
    'bagnet',
    'sop',
]


debug = True
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
# method = 'shap_20'
results_all = {}

for method in methods:

    if method == 'sop':
        explainer = model
    else:
        explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)

    method_list = method.split('_')
    explainer_name = method_list[0]

    if len(method_list) == 2:
        suffix = f'_{method_list[1]}'
    else:
        suffix = ''

    if method != 'sop':
        ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
    else:
        ATTR_VAL_DATA_DIR = None
    # ATTR_VAL_DATA_DIR = None
    
    val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                              split=val_config['evaluation']['split'], 
                                              num_data=val_config['evaluation']['num_data'],
                                              batch_size=val_config['evaluation']['batch_size'],
                                                            attr_dir=ATTR_VAL_DATA_DIR,
                                              processor=processor, debug=debug)

    # for k in tqdm(np.linspace(0.1, 1, 10)):
    k = 0.2
    results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=False, built_in=True)
    results_all[method] = results
    print(results_all[method]['acc'])
    # break

Using cache found in /home/runai-home/.cache/torch/hub/B-cos_B-cos-v2_main


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

...

Loaded 100 images and 100 classes


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
for method in results_all:
    print(method, results_all[method]['acc'])

bcos 0.125
xdnn 0.0
bagnet 0.875
sop 0.6875


In [ ]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc
from sop.tasks.images.imagenet import get_explainer

methods = [
    # 'shap_20',
    # 'rise_20',
    # 'lime_20',
    # 'fullgrad',
    # 'gradcam',
    # 'intgrad',
    # 'attn',
    # 'archipelago',
    # 'mfaba',
    # 'agi',
    # 'ampe',
    'bcos',
    'xdnn',
    'bagnet',
    'sop',
]


debug = True
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
# method = 'shap_20'
results_all = {}

for method in methods:

    if method == 'sop':
        explainer = model
    else:
        explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)

    method_list = method.split('_')
    explainer_name = method_list[0]

    if len(method_list) == 2:
        suffix = f'_{method_list[1]}'
    else:
        suffix = ''

    if method != 'sop':
        ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
    else:
        ATTR_VAL_DATA_DIR = None
    # ATTR_VAL_DATA_DIR = None
    
    val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                              split=val_config['evaluation']['split'], 
                                              num_data=val_config['evaluation']['num_data'],
                                              batch_size=val_config['evaluation']['batch_size'],
                                                            attr_dir=ATTR_VAL_DATA_DIR,
                                              processor=processor, debug=debug)

    # for k in tqdm(np.linspace(0.1, 1, 10)):
    k = 0.2
    results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=False, built_in=True)
    results_all[method] = results
    print(results_all[method]['acc'])
    # break

In [6]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc
from sop.tasks.images.imagenet import get_explainer

methods = [
    # 'shap_20',
    # 'rise_20',
    # 'lime_20',
    # 'fullgrad',
    # 'gradcam',
    # 'intgrad',
    # 'attn',
    # 'archipelago',
    # 'mfaba',
    # 'agi',
    # 'ampe',
    # 'bcos',
    # 'xdnn',
    # 'bagnet',
    'sop',
]


debug = False
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
# method = 'shap_20'
results_all = {}

for method in methods:

    if method == 'sop':
        explainer = model
    else:
        explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)

    method_list = method.split('_')
    explainer_name = method_list[0]

    if len(method_list) == 2:
        suffix = f'_{method_list[1]}'
    else:
        suffix = ''

    # if method != 'sop':
    #     ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
    # else:
    #     ATTR_VAL_DATA_DIR = None
    ATTR_VAL_DATA_DIR = None
    
    val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                              split=val_config['evaluation']['split'], 
                                              num_data=val_config['evaluation']['num_data'],
                                              batch_size=val_config['evaluation']['batch_size'],
                                                            attr_dir=ATTR_VAL_DATA_DIR,
                                              processor=processor, debug=debug)

    # for k in tqdm(np.linspace(0.1, 1, 10)):
    k = 0.2
    results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=True, built_in=True)
    results_all[method] = results
    # break

  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 919 images and 1000 classes


  0%|          | 0/58 [00:00<?, ?it/s]

In [8]:
np.mean(results_all['sop']['acc'])

0.7312295973884657

## Real Run

In [8]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc
from sop.tasks.images.imagenet import get_explainer

methods = [
    # 'shap_20',
    # 'rise_20',
    # 'lime_20',
    # 'fullgrad',
    # 'gradcam',
    # 'intgrad',
    # 'attn',
    # 'archipelago',
    # 'mfaba',
    # 'agi',
    # 'ampe',
    # 'bcos',
    'xdnn',
    'bagnet',
    'sop',
]


debug = False
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
# method = 'shap_20'
results_all = {}

for method in methods:

    if method == 'sop':
        explainer = model
    else:
        explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)

    method_list = method.split('_')
    explainer_name = method_list[0]

    if len(method_list) == 2:
        suffix = f'_{method_list[1]}'
    else:
        suffix = ''

    # if method != 'sop':
    #     ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
    # else:
    #     ATTR_VAL_DATA_DIR = None
    ATTR_VAL_DATA_DIR = None
    
    val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                              split=val_config['evaluation']['split'], 
                                              num_data=val_config['evaluation']['num_data'],
                                              batch_size=val_config['evaluation']['batch_size'],
                                                            attr_dir=ATTR_VAL_DATA_DIR,
                                              processor=processor, debug=debug)

    # for k in tqdm(np.linspace(0.1, 1, 10)):
    k = 0.2
    results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=True, built_in=True)
    results_all[method] = results
    print(method, results_all[method]['acc'])
    # break

  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 919 images and 1000 classes


  0%|          | 0/58 [00:00<?, ?it/s]

xdnn 0.7834602829162133


  0%|          | 0/1001 [00:00<?, ?it/s]

...

bagnet 0.7257889009793254


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 919 images and 1000 classes


  0%|          | 0/58 [00:00<?, ?it/s]

sop 0.7366702937976061


In [9]:
val_config['evaluation']['split']

'val'

In [10]:
results_path = f'imagenet_acc.pt'

torch.save(results_all, results_path)

In [3]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc
from sop.tasks.images.imagenet import get_explainer

methods = [
    # 'shap_20',
    # 'rise_20',
    # 'lime_20',
    # 'fullgrad',
    # 'gradcam',
    # 'intgrad',
    # 'attn',
    # 'archipelago',
    # 'mfaba',
    # 'agi',
    # 'ampe',
    'bcos',
    # 'xdnn',
    # 'bagnet',
    # 'sop',
]


debug = True
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
# method = 'shap_20'
results_all = {}

for method in methods:

    if method == 'sop':
        explainer = model
    else:
        explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)

    method_list = method.split('_')
    explainer_name = method_list[0]

    if len(method_list) == 2:
        suffix = f'_{method_list[1]}'
    else:
        suffix = ''

    if method not in ['sop', 'xdnn', 'bagnet']:
        ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
    else:
        ATTR_VAL_DATA_DIR = None
    # ATTR_VAL_DATA_DIR = None
    
    val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                              split=val_config['evaluation']['split'], 
                                              num_data=val_config['evaluation']['num_data'],
                                              batch_size=val_config['evaluation']['batch_size'],
                                                            attr_dir=ATTR_VAL_DATA_DIR,
                                              processor=processor, debug=debug)

    # for k in tqdm(np.linspace(0.1, 1, 10)):
    k = 0.2
    results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=False, built_in=True)
    results_all[method] = results
    print(method, results_all[method]['acc'])
    # break

Using cache found in /home/runai-home/.cache/torch/hub/B-cos_B-cos-v2_main


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/7 [00:00<?, ?it/s]

bcos 0.125


In [7]:
import numpy as np
from tqdm.auto import tqdm
from sop.metrics import get_acc
from sop.tasks.images.imagenet import get_explainer

methods = [
    # 'shap_20',
    # 'rise_20',
    # 'lime_20',
    # 'fullgrad',
    # 'gradcam',
    # 'intgrad',
    # 'attn',
    # 'archipelago',
    # 'mfaba',
    # 'agi',
    # 'ampe',
    # 'bcos',
    'xdnn',
    # 'bagnet',
    # 'sop',
]


debug = False
k = 0.2

# method = 'lime_20'
# explainer_name = method.split('_')[0]
# method = 'shap_20'
results_all = {}

for method in methods:

    if method == 'sop':
        explainer = model
    else:
        explainer = get_explainer(original_model, backbone_model, method.split('_')[0], device)

    method_list = method.split('_')
    explainer_name = method_list[0]

    if len(method_list) == 2:
        suffix = f'_{method_list[1]}'
    else:
        suffix = ''

    if method not in ['sop']: #, 'xdnn', 'bagnet']:
        ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name}_1_pred{suffix}/val'
    else:
        ATTR_VAL_DATA_DIR = None
    
    val_dataset, val_dataloader = sop.tasks.imagenet.get_dataset(val_config['dataset']['name'], 
                                              split=val_config['evaluation']['split'], 
                                              num_data=val_config['evaluation']['num_data'],
                                              batch_size=val_config['evaluation']['batch_size'],
                                                            attr_dir=ATTR_VAL_DATA_DIR,
                                              processor=processor, debug=debug)

    # for k in tqdm(np.linspace(0.1, 1, 10)):
    k = 0.2
    results = get_acc(val_dataloader, explainer, method, device, k=k, eval_all=False, built_in=True)
    results_all[method] = results
    print(method, results_all[method]['acc'])
    # break

  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 919 images and 1000 classes


  0%|          | 0/58 [00:00<?, ?it/s]

> /shared_data0/weiqiuy/sop/src/sop/metrics/imagenet_acc.py(112)get_acc()
    110 
    111                     # 3
--> 112                     masked_inputs = masks_all[:,None] * inputs
    113                     masked_inputs = ((masked_inputs + 1)/2 - torch.tensor([0.485, 0.456, 0.406])[:, None, None].to(device))\
    114                         / torch.tensor([0.229, 0.224, 0.225])[:, None, None].to(device)



ipdb>  n


...

> /shared_data0/weiqiuy/sop/src/sop/metrics/imagenet_acc.py(124)get_acc()
    122                     masked_inputs = explainer.preprocess(masked_inputs)
    123                 outputs = explainer.model(masked_inputs)
--> 124                 if not isinstance(outputs, torch.Tensor):
    125                     logits = outputs.logits
    126                 else:



ipdb>  outputs.argmax(-1)


tensor([750, 750, 750, 750, 530, 750, 549, 750, 750, 703, 750, 667,  21, 716,
        750, 141], device='cuda:0')


ipdb>  q


BdbQuit: 

In [8]:
explainer_name

'xdnn'

In [14]:
# save_dir = f'/shared_data0/weiqiuy/sop/results/sparsity/{val_config["dataset"]["name"]}/'
# os.makedirs(save_dir, exist_ok=True)

# results_path = f'{save_dir}/{method}.pt'

# torch.save(results, results_path)

# Load

In [1]:
methods = [
    'shap_20',
    'rise_20',
    'lime_20',
    'sop',
    'fullgrad',
    'gradcam',
    'intgrad',
    'attn',
    'archipelago',
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet',
]

In [ ]:
results_all = []
for method in methods:
    save_dir = f'/shared_data0/weiqiuy/sop/results/sparsity/{val_config["dataset"]["name"]}/'
    results_path = f'{save_dir}/{method}.pt'
    results = torch.load(results_path)

In [16]:
save_dir, results_path

('/shared_data0/weiqiuy/sop/results/sparsity/imagenet_s/',
 '/shared_data0/weiqiuy/sop/results/sparsity/imagenet_s//shap_20.pt')

In [17]:
data = torch.load(results_path)

In [18]:
data

{'acc': 0.5625,
 'corrects': [False,
  False,
  False,
  True,
  False,
  True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  False]}